<a href="https://colab.research.google.com/github/amanupg/Machine-Learning/blob/main/Xenia_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tflearn

In [ ]:
!pip install curses

In [ ]:
import numpy
import tflearn
import tensorflow
import random

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [ ]:
import json
with open('xenia_intents.json') as file:
    data = json.load(file)

In [ ]:
words = []
labels = []
docs_x = []  # stores words in each pattern after tokenization
docs_y = []  # stores the corresponding tag

# tokenization
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [ ]:
# stemming and removing duplicates
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

In [ ]:
# one hot encoding
for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1  # set to 1 for the tag the sentence corresponds to

    training.append(bag)
    output.append(output_row)


training = numpy.array(training)
output = numpy.array(output)

In [ ]:
tensorflow.compat.v1.reset_default_graph()

#
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")  # softmax: gives probabilities for each output
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [ ]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [ ]:
def chat():
    print("Bot is active! (type 'quit' to stop)")
    while True:
        inp = input("YOU: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print("BOT: ", random.choice(responses))
        print()

In [ ]:
chat()